In [ ]:
#Open using https://colab.research.google.com/github/MaartenSmeets/insightcode/blob/develop/workshop.ipynb
!git clone https://github.com/MaartenSmeets/insightcode.git
%cd insightcode
!git checkout develop

In [ ]:
#Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
#Start server
get_ipython().system_raw('ollama serve &')

In [ ]:
#Install requirements
!pip install -r requirements.txt

In [ ]:
# Download model with progress monitoring
# This can be done locally with. ollama pull deepseek-coder-v2:16b-lite-instruct-q5_K_M
import subprocess
import re

def pull_model_with_progress(model_name):
    """Pull a model using ollama and monitor progress."""
    print(f"Start downloading model: '{model_name}'.")
    try:
        # Start the ollama pull command
        process = subprocess.Popen(
            ["ollama", "pull", model_name],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,  # Combine stderr with stdout
            universal_newlines=True,   # Ensure text mode for Python 3
            bufsize=1                  # Line-buffered
        )

        current_percentage = None
        reached_100_percent = False  # Flag to stop displaying progress after 100%

        # Read the output line by line
        for line in iter(process.stdout.readline, ''):
            # Remove any extra whitespace
            line = line.strip()
            # Use regex to find percentages in the output
            match = re.search(r'(\d+)%', line)
            if match:
                percentage = match.group(1)
                if percentage != current_percentage:
                    current_percentage = percentage
                    if not reached_100_percent:
                        print(f"Downloading... {percentage}%")
                    if percentage == '100':
                        reached_100_percent = True  # Stop displaying progress after 100%
            elif "Downloaded" in line:
                # Handle completion message
                print("Download complete.")
            elif "Error" in line or "Failed" in line:
                # Handle error messages
                print(f"Error: {line}")
                break

        # Wait for the process to finish
        process.wait()

        # Check if the command was successful
        if process.returncode == 0:
            print(f"Model '{model_name}' downloaded successfully.")
        else:
            print(f"Failed to download model '{model_name}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Use the function to download the model
pull_model_with_progress("deepseek-coder-v2:16b-lite-instruct-q5_K_M")

In [ ]:
#Put this repository in the repo folder for analysis
!mkdir repo
!find . -path './.venv' -prune -o -type f -name '*.py' -exec install -D {} repo/{} \;

In [ ]:
#Execute!
!python main.py